## Some experimentation with API using KerbalX

In [ ]:
import krpc
import math
import time

In [ ]:
conn = krpc.connect(name='Main')
KSC = conn.space_center
vessel = KSC.active_vessel
print("Main is online!")

In [ ]:
ap = vessel.auto_pilot
vc = vessel.control

#### Setting up streams for telemetry

In [ ]:
# telemetry setup
ut = conn.add_stream(getattr, conn.space_center, 'ut')
altitude = conn.add_stream(getattr, vessel.flight(), 'mean_altitude')
apoapsis = conn.add_stream(getattr, vessel.orbit, 'apoapsis_altitude')
s2_resources = vessel.resources_in_decouple_stage(stage=2, cumulative=False)
s3_resources = vessel.resources_in_decouple_stage(stage=3, cumulative=False)
s2_srb_fuel = conn.add_stream(stage_2_resources.amount, 'SolidFuel')
s3_srb_fuel = conn.add_stream(stage_3_resources.amount, 'SolidFuel')

#### Gravity turn function from example in kRPC docs
* Added a few helper functions

In [ ]:
def G_turn(turn_start,turn_end,tgt_alt):
    """ Calculates the pitch, heading, and roll relative to the current celestial body surface"""
    turn_start = 250
    turn_end = 45000
    tgt_alt = 150000
    v_ref = vessel.reference_frame
    s_ref = vessel.surface_reference_frame
    v_dir = vessel.direction(s_ref)
    h_dir = (0, v_dir[1], v_dir[2])

    def V_mag(v):
        """ """
        mag = math.sqrt((v[0]) ** 2 + (v[1]) ** 2 + (v[2]) ** 2)
        return mag

    def dot_p(u, v):
        """ method 1 for dot product calculation to find angle  """
        return u[0]*v[0] + u[1]*v[1] + u[2]*v[2]

    def cross_p(u, v):
        """ returns the cross product of two vectors u x v - orthogonal to u and v """
        return (u[1]*v[2] - u[2]*v[1],
                u[2]*v[0] - u[0]*v[2],
                u[0]*v[1] - u[1]*v[0])

    def UV_theta(u, v):
        """ returns angle between vectors, checks if they are parallel """
        dotp = dot_p(u, v)
        if dotp == 0:
            return 0
        else:
            theta = math.acos(dotp / (V_mag(u) * V_mag(v))) * (180/math.pi)
            return theta
    #......................................   main  .........................................................#

    # pitch definition
    pitch = UV_theta(v_dir, h_dir)
    if v_dir[0] < 0:
        pitch = -pitch
    # heading definition
    north = (0, 1, 0)
    heading = UV_theta(north, h_dir)
    if h_dir[2] < 0:
        heading = 360 - heading
    # roll definition
    up = (1, 0, 0)
    norm = cross_p(v_dir, up)
    # define up
    v_up = conn.space_center.transform_direction((0, 0, -1), v_ref, s_ref)
    # angle between up and normal
    roll = UV_theta(up, norm)
    if up[0] > 0:
        roll *= -1
    elif roll < 0:
        roll += 180
    else:
        roll -= 180
    # comment out if no printing is desired
    # print('pitch = % 5.1f, heading = % 5.1f, roll = % 5.1f' %(pitch, heading, roll))
    return pitch,heading,roll

#### Checking out pitch heading and roll

In [ ]:
pitch = round(G_turn(250,45000,150000)[0],1)
heading = round(G_turn(250,45000,150000)[1],1)
roll = round(G_turn(250,45000,150000)[2],1)
print(pitch,heading,roll)

#### Returns the atmospheric depth of body for the current orbit

In [ ]:
depth = vessel.orbit.body.atmosphere_depth
depth